## Implementeer elk gemaakt ETL-schema in Python

**imports & connection**

In [2]:
import pandas as pd
import pyodbc

In [3]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'United_outdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor   

**Database connection**

In [4]:
Adventure = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'AdventureWorks2019'
}


Northwind = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'Northwind'
}

access_db_path = r'C:\Users\Humberto de Castro\OneDrive\Desktop\SEM4\AenC\aenc.accdb'

#Connect to AdventureWorks
Adventure_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Adventure['servername']};DATABASE={Adventure['database']};Trusted_Connection=yes;")
Adventure_cursor = Adventure_conn.cursor()

#Connect to Northwind
Northwind_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Northwind['servername']};DATABASE={Northwind['database']};Trusted_Connection=yes;")
Northwind_cursor = Northwind_conn.cursor()

#Connect to AenC
AenC_conn = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};")



In [5]:
AdventureWorks_queries = {
    'Shipmethod' : 'SELECT * FROM Purchasing.Shipmethod'
}

Northwind_queries = {
    'Shippers' : 'SELECT * FROM Shippers'
}

AenC_queries = {
    'sales_order' : 'SELECT * FROM sales_order',
    'sales_order_item' : 'SELECT * FROM sales_order_item',
    'Product' : 'SELECT * FROM Product' 
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, AenC_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_53564\2043654250.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_53564\2043654250.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_53564\2043654250.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table

## Ship

Transform

In [6]:
Shippers.rename(columns={
    'ShipperID': 'ShipMethodID',
    'CompanyName' : 'Name'
}, inplace=True)

In [7]:
Shipmethod['ShipMethodID'] = Shipmethod['ShipMethodID'].apply(lambda x: f'AW_{x}')
Shippers['ShipMethodID'] = Shippers['ShipMethodID'].apply(lambda x: f'NW_{x}')
# AenC_Order heeft geen ShipMethodID

In [9]:
# Zorg ervoor dat alle vereiste kolommen bestaan en selecteer ze
required_columns = ['ShipMethodID', 'Name', 'ShipBase', 'ShipRate']

for col in required_columns:
    if col not in Shippers.columns:
        Shippers[col] = None
    if col not in Shipmethod.columns:
        Shipmethod[col] = None

# Selecteer de kolommen in de juiste volgorde
Shippers = Shippers[required_columns]
Shipmethod = Shipmethod[required_columns]

# Combineer de dataframes
Ship = pd.concat([Shipmethod, Shippers], ignore_index=True)

print(Ship.dtypes)

ShipMethodID     object
Name             object
ShipBase        float64
ShipRate        float64
dtype: object


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_53564\3011861440.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  Ship = pd.concat([Shipmethod, Shippers], ignore_index=True)


### Load in ssms

In [11]:
# Insert data into the Ship table
for index, row in Ship.iterrows():
    try:
        query = """
        INSERT INTO [Ship] (ShipMethodID, Name, ShipBase, ShipRate)
        VALUES (?, ?, ?, ?)
        """
        # Ensure values are in the correct format
        params = (
            row['ShipMethodID'],
            row['Name'],
            float(row['ShipBase']) if pd.notnull(row['ShipBase']) else None,
            float(row['ShipRate']) if pd.notnull(row['ShipRate']) else None
        )
        export_cursor.execute(query, params)
    except pyodbc.Error as e:
        print(f"An error occurred: {e}")
        print(query)
        print("Parameters:", params)

export_conn.commit()